In [1]:
# Cell 1 - setup
import requests
import pandas as pd
from IPython.display import display, Markdown

BASE_URL = "http://localhost:3000/api"  # change if needed

# Cell 2 - GET all employees
def get_all_employees():
    url = f"{BASE_URL}/employees"
    resp = requests.get(url, timeout=5)
    resp.raise_for_status()
    data = resp.json()
    # If your API returns {status, count, data}, adapt as needed
    if isinstance(data, dict) and "data" in data:
        records = data["data"]
    else:
        records = data
    df = pd.DataFrame(records)
    return df


In [2]:
get_all_employees()

,id,name,unit,attendance,years_of_service
0,1,Sandy Supriyono,Ho Network Specialist,bekerja,3
1,2,Mirpan Munandar,Ho Network Specialist,bekerja,3
2,3,Rhevo Ajjunda Rakhman,Branches Network Specialist,bekerja,7
3,4,Fitra Nopri Setiawan,Branches Network Specialist,bekerja,7
4,5,Jufri Pramono,Echannels Network Specialist,cuti,4
...,...,...,...,...,...
67,68,Anggito Anju Hartawan Manalu,Front End & Mobile App Specialist,cuti,9
68,69,Avisenna Abimanyu,Digital Back End Specialist,bekerja,4
69,70,Wahyu Dwinanto,Branch End Point Specialist,bekerja,3
70,71,Kevin Vasco Aruji,Ho End Point Specialist,cuti,4


In [8]:
# --- Ambil data pegawai ---
df_emp = get_all_employees()

df_emp["years_of_service"] = pd.to_numeric(df_emp["years_of_service"], errors="coerce")

# --- Hitung kuartil ---
q1 = df_emp["years_of_service"].quantile(0.25)
q2 = df_emp["years_of_service"].quantile(0.50)
q3 = df_emp["years_of_service"].quantile(0.75)

print("Q1:", q1, "| Q2:", q2, "| Q3:", q3)

# --- Fungsi bobot seniority (maksimum 1) ---
def seniority_weight(y):
    if pd.isna(y):
        return None
    if y <= q1: 
        return 0.25
    elif y <= q2:
        return 0.50
    elif y <= q3:
        return 0.75
    else:
        return 1.0

# Hitung bobot
df_emp["seniority_weight"] = df_emp["years_of_service"].apply(seniority_weight)

display(df_emp[["name", "years_of_service", "seniority_weight"]])


Q1: 4.0 | Q2: 5.0 | Q3: 8.0


,name,years_of_service,seniority_weight
0,Sandy Supriyono,3,0.25
1,Mirpan Munandar,3,0.25
2,Rhevo Ajjunda Rakhman,7,0.75
3,Fitra Nopri Setiawan,7,0.75
4,Jufri Pramono,4,0.25
...,...,...,...
67,Anggito Anju Hartawan Manalu,9,1.00
68,Avisenna Abimanyu,4,0.25
69,Wahyu Dwinanto,3,0.25
70,Kevin Vasco Aruji,4,0.25
